In [1]:
import semiolog as slg

In [2]:
semiotic = slg.Cenematic("en_bnc_old_segments")

Downloading: 100%|██████████| 511M/511M [00:08<00:00, 67.0MB/s] 
All model checkpoint layers were used when initializing TFBertForMaskedLM.

All the layers of TFBertForMaskedLM were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.
All model checkpoint layers were used when initializing TFBertForMaskedLM.

All the layers of TFBertForMaskedLM were initialized from the model checkpoint at models/en_bnc_old_segments/paradigms/tf_model.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.
Using custom data configuration corpus-48c43d9b917d9e96
Reusing dataset text (/Users/Gianni/.cache/huggingface/datasets/text/corpus-48c43d9b917d9e96/0.0.0/e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5)
100%|████

In [3]:
import tensorflow as tf

In [51]:
sent = semiotic.corpus.train["text"][10000]
sent = "Whenever it is possible bring your own lunch to the office"
sent

'Whenever it is possible bring your own lunch to the office'

In [52]:
sent_len = len(sent.split())
sent_mask = [" ".join([t if i!=n else "[MASK]" for i,t in enumerate(sent.split())]) for n in range(sent_len)]

In [53]:
# input = semiotic.paradigmatic.bert_tokenizer(sent_mask)
# outputs = semiotic.paradigmatic.model(input["input_ids"])

input = semiotic.paradigmatic.hf_tokenizer(sent_mask, return_tensors="tf")
outputs = semiotic.paradigmatic.hf_model(input)

In [54]:
# Gather all the masked logits in a batch, by taking the row diagonal of the 3D tensor

# parad_logits = tf.gather_nd(outputs.logits, indices=[[i,i] for i in range(sent_len)])
parad_logits = tf.gather_nd(outputs.logits[:,1:-1], indices=[[i,i] for i in range(sent_len)])

In [55]:
logits_positif = tf.nn.relu(parad_logits)

In [56]:
probs, norms = tf.linalg.normalize(logits_positif, ord=1, axis=1)

In [57]:
# Maybe its cheaper to compute top k for k = non_zero, which would differ from row to row, and hence not yeld a tensor. To be tested.

non_zeroes = tf.math.count_nonzero(probs,axis=1)
max_non_zeroes = max(non_zeroes).numpy()

parad_data = tf.math.top_k(probs, k = max_non_zeroes, sorted=True)

In [58]:
parad_tokens = [semiotic.syntagmatic.tokenizer.decode(parad.numpy()[:parad_len]).split() for parad, parad_len in zip(parad_data.indices, non_zeroes)]

parad_tokens = [semiotic.paradigmatic.hf_tokenizer.decode(parad.numpy()[:parad_len]).split() for parad, parad_len in zip(parad_data.indices, non_zeroes)]

In [59]:
[p[:5] for p in parad_tokens]

[['if', '"', "where'but", 'when', 'so'],
 ['it', 'that', 'this', 'anything', 'else'],
 ['is', 'becomes', 'was', 'be', 'seems'],
 [',', 'you', 'please.', 'just', 'then'],
 ['bring', 'take', 'deliver', 'send', 'get'],
 ['your', 'their', 'our', 'my', 'you'],
 ['own', 'free', 'daily', 'hot', 'usual'],
 ['staff', 'books', 'computer', 'car', 'secretary'],
 ['to', 'into', 'from', 'at', 'in'],
 ['the', 'my', 'our', 'your', 'this'],
 ['table', 'restaurant', 'cafe', 'office', 'cafeteria.']]

In [61]:
max_parad = 100 # max_non_zeroes
slg.util.plot_scatter_line(list(range(max_parad)),parad_data.values[0],sent.split()[0], add_trace=[(list(range(max_parad)), v, t) for v, t in zip(parad_data.values[1:],sent.split()[1:])])

In [34]:
non_zeroes

<tf.Tensor: shape=(6,), dtype=int64, numpy=array([ 544,  604,  161, 1994,  749,  140])>